In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
from numpy import genfromtxt
from sklearn.cluster import KMeans
from sklearn import svm

In [2]:
class SOM(object):
    def __init__(self, m, n):
        self.net_dim = np.array([m, n])
        self.weights = None
        self.hit_map = np.zeros([self.net_dim[0], self.net_dim[1]])
        self.init_radius = np.max(self.net_dim)/2
        self.umatrix = np.zeros([self.net_dim[0], self.net_dim[1]])
        self.umat_2 = np.zeros([self.net_dim[0]*2 - 1, self.net_dim[1]*2 - 1])
        self.bmu_list = None
        self.label_map = np.zeros([self.net_dim[0], self.net_dim[1]]) + -1
        self.classif_log=None # matrix for storing data for each neu, #times it becomed BMU for each class
        self.neu_class=np.zeros([m*n])# store class for each neuron 
        self.trainAccuracy=None
        self.testAccuracy=None

    def fit_model(self, train_x, train_y=None, init_lr=0.1, epochs=100):
        # Initializing the weight matrix
        self.weights = np.random.random((self.net_dim[0], self.net_dim[1], train_x.shape[1]))
        self.bmu_list = np.zeros(train_x.shape[0])
        time_constant = epochs/np.log(self.init_radius)
        
        if not (train_y is None):
            #print("np.unique(train_y).shape[0]",np.unique(train_y).shape[0])
            self.classif_log=np.zeros([np.unique(train_y).shape[0] , self.net_dim[0]*self.net_dim[1]])

        for epoch in range(epochs):
            print('Processing epoch:{}'.format(epoch))
            """decay the radius and the learning rate"""
            if epoch != 0:
                radius = SOM.decay_radius(self.init_radius,epoch, time_constant)
                lr = SOM.decay_learning_rate(init_lr, epoch, epochs)

            else:
                radius = self.init_radius
                lr = init_lr

            #print("train_x.shape[0]",train_x.shape[0])            
            randlist=random.sample(range(train_x.shape[0]), train_x.shape[0])
            for t in range(train_x.shape[0]):

                # Picking a random pattern
                r = randlist[t]                
                # Finding the bmu for the random pattern
                bmu_index, bmu = self.find_bmu(train_x[r, :])
                #print("bmu_index : ",bmu_index) 
                self.bmu_list[r] = bmu_index[0]*self.weights.shape[1] + bmu_index[1]
                #print("self.bmu_list[r] :: ",self.bmu_list[r])

                # updating the weights
                for i in range(self.net_dim[0]):
                    for j in range(self.net_dim[1]):
                        w = self.weights[i, j, :]
                        w_bmu_dist = SOM.get_distance(w, bmu)

                        h = self.get_gaussian_membership(w_bmu_dist, radius)
                        delta_w = (lr * h * (train_x[r, :] - w))
                        new_w = w + delta_w
                        self.weights[i, j, :] = new_w                        
                        if np.isnan(self.weights).any():
                            print(' weights NaN at epoch {}'.format(epoch))

                            return
                        # print('\n')
            ##
            if not (train_y is None) :       
                for t in range(train_x.shape[0]):
                    bmu_index, bmu = self.find_bmu(train_x[t, :])
                    self.bmu_list[t] = bmu_index[0]*self.weights.shape[1] + bmu_index[1]
                    #print("self.bmu_list[r]",self.bmu_list[r])
                    self.classif_log[int(train_y[t]),int(self.bmu_list[t])]=(self.classif_log[int(train_y[t]),int(self.bmu_list[t])])+1
                print("Train Accuracy",(sum(self.classif_log.max(axis=0))*100)/ (train_x.shape[0]) )
                self.classif_log=np.zeros([np.unique(train_y).shape[0] , self.net_dim[0]*self.net_dim[1]])
                self.trainAccuracy=(sum(self.classif_log.max(axis=0))*100)/ (train_x.shape[0])
 
        if not (train_y is None) :       
            for t in range(train_x.shape[0]):
                bmu_index, bmu = self.find_bmu(train_x[t, :])
                self.bmu_list[t] = bmu_index[0]*self.weights.shape[1] + bmu_index[1]
                #print("self.bmu_list[r]",self.bmu_list[r])
                """                    
                print("bla", bmu_index,bmu ," t",t) # bla [0, 2] [-0.002496    0.00031421 -0.00322767] print("Train y val :",int(train_y[t]))
                print("self.bmu_list :",int(self.bmu_list[t]))
                """
                #print("int(train_y[t])",int(train_y[t]) )
                #print("int(self.bmu_list[t])",int(self.bmu_list[t]))
                self.classif_log[int(train_y[t]),int(self.bmu_list[t])]=(self.classif_log[int(train_y[t]),int(self.bmu_list[t])])+1
                        
            print("Check \n",self.classif_log)        
            self.neu_class=self.classif_log.argmax(axis=0) 
            print("self.neu_class",self.neu_class)
            print("Train Accuracy",(sum(self.classif_log.max(axis=0))*100)/ (train_x.shape[0]) )
            self.trainAccuracy=(sum(self.classif_log.max(axis=0))*100)/ (train_x.shape[0])
        self.set_umatrix()
        self.set_hitmap()
        #now test for accuracy
        #if train_y is not None:
            #self.set_label_map(train_y)#this needs checking cz it prints something 
        
    def test_model(self, test_x, test_y=None):
        
        correct=0;
        
        for t in range(test_x.shape[0]):
            bmu_index, bmu = self.find_bmu(test_x[t, :])
            neu_no=bmu_index[0]*self.weights.shape[1] + bmu_index[1]
            #print("neu_no : ",neu_no)
            class_label=self.neu_class[neu_no]

            if(class_label==test_y[t]):
                correct=correct+1
        print("Test Accuracy : ", (correct*100/test_x.shape[0]))
        self.testAccuracy=(correct*100/test_x.shape[0])



    def find_bmu(self, x):
        min_dist = float('inf')
        bmu_index = [-1, -1]
        bmu = self.weights[0, 0, :]
        for i in range(self.net_dim[0]):
            for j in range(self.net_dim[1]):
                w = self.weights[i, j, :]
                d = SOM.get_distance(x, w)
                #print("d : ",d)
                if d < min_dist:
                    min_dist = d
                    bmu_index = [i, j]
                    bmu = w

        return bmu_index, bmu
    
    def find_bmu_list(self, x):
        min_dist = float('inf')
        bmu_index = [-1, -1]
        bmu = self.weights[0, 0, :]
        dis_list=[]
        for i in range(self.net_dim[0]):
            for j in range(self.net_dim[1]):
                w = self.weights[i, j, :]
                d = SOM.get_distance(x, w)
                dis_list.append(d)
                #print("d : ",d)
                if d < min_dist:
                    min_dist = d
                    bmu_index = [i, j]
                    bmu = w
        s = dis_list
        sorted_list=sorted(range(len(s)), key=lambda k: s[k])
        #print(dis_list)
        #print(bmu_index)
        #print(dis_list[sorted_list[0]])
        #print(min_dist)
        return bmu_index, bmu, sorted_list

    def get_gaussian_membership(self, squared_distance_from_bmu, radius):
        h = np.exp(-squared_distance_from_bmu/(2*(radius**2)))
        # h = np.exp(squared_distance_from_bmu/(2*(radius**2)))
        # print('h:{}'.format(h))
        return h

    def set_hitmap(self):
        unique, counts = np.unique(self.bmu_list, return_counts=True)
        #print("unique",unique)
        #print("counts",counts)
        bmu_dict = dict(zip(unique.astype(int), counts))

        for bmu, count in bmu_dict.items():
            i = int(bmu / self.label_map.shape[1])
            j = bmu % self.label_map.shape[1]
            self.hit_map[i, j] = count
            
        #print(np.shape(self.hit_map))

    def set_label_map(self, train_y):
        # print(self.label_map)

        unique, counts = np.unique(self.bmu_list, return_counts=True)
        bmu_dict = dict(zip(unique.astype(int), counts))
        print(bmu_dict)

        for bmu, count in bmu_dict.items():
            # print('bmu: {}, count: {}'.format(bmu, count))
            # print(self.bmu_list[self.bmu_list == bmu])
            idx = np.where(self.bmu_list == bmu)
            unique_labels, label_counts = np.unique(train_y[idx], return_counts=True)

            labels_dict = dict(zip(unique_labels.astype(int), label_counts))
            # print(labels_dict)

            import operator
            # print(max(labels_dict.items(), key=operator.itemgetter(1))[0])

            i = int(bmu / self.label_map.shape[1])
            j = bmu % self.label_map.shape[1]

            self.label_map[i, j] = max(labels_dict.items(), key=operator.itemgetter(1))[0]

    @classmethod
    def get_distance(cls, x1, x2):
        return np.sum((x1 - x2) ** 2)

    @classmethod
    def decay_radius(cls, init_radius, e, time_constant):
        return init_radius * np.exp(-e / time_constant)

    @classmethod
    def decay_learning_rate(cls, init_lr, e, num_epochs):
        return init_lr * np.exp(-e / num_epochs)

    @classmethod
    def draw_square(cls, x, y, dim, color):
        rect = plt.Rectangle((x, y), dim, dim, fc=color)

        return rect

    def show_hitmap(self, show_hits=False):
        print("show hit map")
        #fig, ax = plt.subplots()
        plt.figure()
        plt.imshow(self.hit_map, cmap='jet')
        plt.title('Hit map')
        plt.colorbar()        
        if show_hits:
            for i in range(self.net_dim[0]):
                for j in range(self.net_dim[1]):
                    c = self.hit_map[j,i]
                    plt.text(i, j, str(c), va='center', ha='center')    
        plt.show()
        
    def show_umatrix(self):
        plt.figure()
        plt.imshow(self.umatrix, cmap='jet')
        plt.title('U-Matrix')
        plt.colorbar()
        plt.draw()
        plt.show()

    def set_umatrix(self, nbhd='neumann'):
        for i in range(self.weights.shape[0]):
            for j in range(self.weights.shape[1]):
                self.umatrix[i, j] = self.calculate_average_distance([i, j], nbhd=nbhd)

    def calculate_average_distance(self, index, nbhd='neumann'):
        avg_distance = -1

        if nbhd == 'neumann':
            n = np.zeros([4])
            div = 4
            x = index[0]
            y = index[1]
            # Above
            if (x - 1) >= 0:
                n[0] = np.sqrt(SOM.get_distance(self.weights[x - 1, y, :], self.weights[x, y, :]))
            else:
                div -= 1
            # Below
            if (x + 1) < self.weights.shape[0]:
                n[1] = np.sqrt(SOM.get_distance(self.weights[x + 1, y, :], self.weights[x, y, :]))
            else:
                div -= 1
            # Left
            if (y - 1) >= 0:
                n[2] = np.sqrt(SOM.get_distance(self.weights[x, y - 1, :], self.weights[x, y, :]))
            else:
                div -= 1
            # Right
            if (y + 1) < self.weights.shape[1]:
                n[3] = np.sqrt(SOM.get_distance(self.weights[x, y + 1, :], self.weights[x, y, :]))
            else:
                div -= 1

            avg_distance = np.sum(n) / div
        elif nbhd == 'moore':
            # TODO implement for moore neighborhood
            pass

        else:
            print('Not a valid Neighborhood')

        return avg_distance

    def set_umat_2(self):
        # TODO: Need to optimize this code
        r = 0
        for i in range(self.net_dim[0]):
            c = 0
            for j in range(self.net_dim[1]):
                if c+1 < self.umat_2.shape[1]:
                    self.umat_2[r, c+1] = SOM.get_distance(self.weights[i, j, :], self.weights[i, j+1, :])

                if r+1 < self.umat_2.shape[0]:
                    self.umat_2[r+1, c] = SOM.get_distance(self.weights[i, j, :], self.weights[i+1, j, :])
                c = c+2
            r = r+2

        temp = np.zeros([self.umat_2.shape[0], self.umat_2.shape[1], self.weights.shape[2]])
        r = 0
        for i in range(0, self.umat_2.shape[0], 2):
            c = 0
            for j in range(0, self.umat_2.shape[1],2):
                temp[i, j] = self.weights[r, c, :]

                c += 1
            r += 1

        for i in range(self.umat_2.shape[0]):
            for j in range(self.umat_2.shape[1]):
                if i % 2 != 0 and j % 2 != 0:
                    self.umat_2[i, j] = (SOM.get_distance(temp[i-1, j-1, :], temp[i+1, j+1, :]) +
                                         SOM.get_distance(temp[i-1, j+1, :], temp[i+1, j-1, :]))/2

    def show_umatrx_2(self, size=2):
        max_dist = np.max(self.umat_2)
        print(max_dist)
        cmap = matplotlib.pyplot.get_cmap('jet', lut=256)
        plt.figure()
        plt.imshow(self.umat_2, cmap='jet')
        plt.title('U-Matrix')
        plt.colorbar()
        plt.draw()
        plt.show()
        
    def print_save_weights(self,fileName):
        """
        for i in range(self.weights.shape[0]):
            print(self.weights[i,:]) 
        """  
        np.save(fileName,self.weights)
        #print("load: \n", np.load('maximums.npy'))

def standardize_data(X):
    x_norm = (X - X.mean(axis=0)) / X.std(axis=0)
    return x_norm
    from tempfile import TemporaryFile   


# Load train data

In [3]:
dataset_name='TrainSet_Processed.csv'
data1=genfromtxt(dataset_name,delimiter=",")
print(np.shape(data1))

(125973, 44)


In [4]:
dataset_tst='TestSet_Processed.csv'
data2=genfromtxt(dataset_tst,delimiter=",")
print(np.shape(data2))

(22543, 44)


# Train Data Selection

In [80]:
#train
ii = np.where(data1[:,43] == 0)[0]
train_x=(data1[ii, 0:41])

print(np.shape(ii))
print(np.shape(train_x))

ii = np.where(data1[:,41] == 1)[0]
train_x_attack=(data1[ii, 0:41])

print(np.shape(ii))
print(np.shape(train_x_attack))
print(np.shape(data1))

(67343,)
(67343, 41)
(41214,)
(41214, 41)
(125973, 44)


In [81]:
ii = np.where(data2[:,43] == 0)[0]
test_x=(data2[ii, 0:41])

print(np.shape(ii))
print(np.shape(test_x))

ii = np.where(data2[:,41] == 1)[0]
test_x_attack=(data2[ii, 0:41])

print(np.shape(ii))
print(np.shape(test_x_attack))
print(np.shape(data2))

(9710,)
(9710, 41)
(4657,)
(4657, 41)
(22543, 44)


In [7]:
#normalize the data

# Train SOM

In [191]:
som_dim=8
som = SOM(som_dim, som_dim)
som.fit_model(train_x, epochs=5)

Processing epoch:0
Processing epoch:1
Processing epoch:2
Processing epoch:3
Processing epoch:4


In [194]:
filename_for_weight='weight1'
som.print_save_weights(filename_for_weight)
filename_for_weight=filename_for_weight+'.npy'
print(filename_for_weight)
points=np.load(filename_for_weight)
print(np.shape(points))

weight1.npy
(8, 8, 41)


In [195]:
BMUList=[]
for i in range(0,som_dim):
    for j in range(0,som_dim):        
        BMUList.append(points[i,j,:])
        
print(np.shape(BMUList))

(64, 41)


# Train KMeans

In [196]:
no_clusters=3
kmeans = KMeans(n_clusters=no_clusters, random_state=0).fit(BMUList)
labels_of_BMUs=kmeans.labels_
print(np.shape(labels_of_BMUs))
print(labels_of_BMUs)

(64,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [197]:
#for each data point store the cluster label
data_cluster_list=np.zeros(np.shape(train_x)[0]);
for i in range(0,np.shape(train_x)[0]):
    bmu_index, bmu = som.find_bmu(train_x[i, :])    
    data_cluster_list[i]=(labels_of_BMUs[((bmu_index[0])*som_dim) +bmu_index[1]]);
print(data_cluster_list) 
print(type(data_cluster_list))

#check this code 

[ 0.  0.  0. ...,  0.  0.  0.]
<class 'numpy.ndarray'>


In [198]:
#seperate train data for k clusters 

datasets=[];
for i in range(0,no_clusters):    
    searchval = i
    ii = np.where(data_cluster_list == i)[0]
    print('________________________')
    #print(ii)
    datasets.append(train_x[ii, :])
    print(np.shape(datasets[i]))

________________________
(67331, 41)
________________________
(5, 41)
________________________
(7, 41)


# Ensemble OCSVM

In [199]:
OCC_list = list()
 
#create one class SVM for each data cluster
for i in range(no_clusters):
    OCC_list.append(svm.OneClassSVM(nu=0.1, kernel='poly',degree=2, gamma=0.1))
    #OCC_list.append(svm.OneClassSVM(nu=0.01, kernel="rbf", gamma=0.1))
#OCC_list.append(svm.OneClassSVM(nu=0.1, kernel='poly',degree=2, gamma=0.1))
#OCC_list.append(svm.OneClassSVM(nu=0.2, kernel='poly',degree=2, gamma=0.1))
#OCC_list.append(svm.OneClassSVM(nu=0.2, kernel='poly',degree=2, gamma=0.1))
for i in range(no_clusters):
    print(i)
    OCC_list[i].fit(datasets[i])
    print(np.shape(datasets[i]))
    
print(OCC_list[1])

0
(67331, 41)
1
(5, 41)
2
(7, 41)
OneClassSVM(cache_size=200, coef0=0.0, degree=2, gamma=0.1, kernel='poly',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)


In [200]:
#normal Data : train
data_set=train_x
dim=np.shape(data_set)
error_count=0;
for i in range(dim[0]):
    bmu_index, bmu, sorted_list=som.find_bmu_list(data_set[i,:])
    n=[data_cluster_list[i] for i in (sorted_list[0:10])]
    classfier_list=np.unique(n)
    #print(classfier_list)
    flag=1
    for classifier_id in (classfier_list):
        #print(classifier_id)
        y_pred_train = OCC_list[int(classifier_id)].predict(data_set[i,:].reshape(1, -1))
        #print(y_pred_train)
        n_error_train = y_pred_train[y_pred_train == 1].size
        #print(n_error_train)
        #print('correct')
        if n_error_train!=0:
            flag=0
            #print('correct')
    
    if flag==1:
        error_count=error_count+1
        #print('error')
    #print('---------------------')
    
print('error',(error_count*100/dim[0]))  

error 9.99509971340748


In [201]:
# test with normal data in the test set
data_set=test_x
dim=np.shape(data_set)
error_count=0;
for i in range(dim[0]):
    bmu_index, bmu, sorted_list=som.find_bmu_list(data_set[i,:])
    n=[data_cluster_list[i] for i in (sorted_list[0:10])]
    classfier_list=np.unique(n)
    #print(classfier_list)
    flag=1
    for classifier_id in (classfier_list):
        #print(classifier_id)
        y_pred_train = OCC_list[int(classifier_id)].predict(data_set[i,:].reshape(1, -1))
        #print(y_pred_train)
        n_error_train = y_pred_train[y_pred_train == 1].size
        #print(n_error_train)
        #print('correct')
        if n_error_train!=0:
            flag=0
            #print('correct')
    
    if flag==1:
        error_count=error_count+1
        #print('error')
    #print('---------------------')

error_count1=error_count
print('error',(error_count*100/dim[0])) 

error 5.901132852729146


In [202]:
# test with attack data in the test set
data_set=test_x_attack
dim=np.shape(data_set)
error_count=0;
for i in range(dim[0]):
    bmu_index, bmu, sorted_list=som.find_bmu_list(data_set[i,:])
    n=[data_cluster_list[i] for i in (sorted_list[0:10])]
    classfier_list=np.unique(n)
    #print(classfier_list)
    flag=1
    for classifier_id in (classfier_list):
        #print(classifier_id)
        y_pred_train = OCC_list[int(classifier_id)].predict(data_set[i,:].reshape(1, -1))
        #print(y_pred_train)
        n_error_train = y_pred_train[y_pred_train == 1].size
        #print(n_error_train)
        if n_error_train!=0:
            flag=0
            #print('correct')
    
    if flag==0:
        error_count=error_count+1
        #print('error')
    #print('---------------------')
error_count2=error_count   
print('error',(error_count*100/dim[0])) 

error 3.2209576980888985


In [203]:
# test with attack data in the train set
data_set=train_x_attack
dim=np.shape(data_set)
error_count=0;
for i in range(dim[0]):
    bmu_index, bmu, sorted_list=som.find_bmu_list(data_set[i,:])
    n=[data_cluster_list[i] for i in (sorted_list[0:10])]
    classfier_list=np.unique(n)
    #print(classfier_list)
    flag=1
    for classifier_id in (classfier_list):
        #print(classifier_id)
        y_pred_train = OCC_list[int(classifier_id)].predict(data_set[i,:].reshape(1, -1))
        #print(y_pred_train)
        n_error_train = y_pred_train[y_pred_train == 1].size
        #print(n_error_train)
        if n_error_train!=0:
            flag=0
            #print('correct')
    
    if flag==0:
        error_count=error_count+1
        #print('error')
    #print('---------------------')
error_count3=error_count    
print('error',(error_count*100/dim[0])) 

error 4.8138981899354585


In [204]:
print('total test error',((error_count1+error_count2+error_count3)*100/(np.shape(test_x)[0]+np.shape(test_x_attack)[0]+np.shape(train_x_attack)[0]))) 

total test error 4.87036937082816


In [205]:
print('test Accuracy: ', 100-((error_count1+error_count2+error_count3)*100/(np.shape(test_x)[0]+np.shape(test_x_attack)[0]+np.shape(train_x_attack)[0])))

test Accuracy:  95.12963062917184


In [ ]:
4.868570194850759

In [ ]:
95.13142980514924